# Overview

Here, we will demonstrate how to carry out geometry optimizations using ASE. For this, we need to pick an energy model --- which in ASE is called a calculator. The calculator is the simulation engine that, given an atomic configuration, will return the energy of the system (as well as forces and, for solids, the unit cell stresses).

For this exercise, we will mainly use a "machine learning potential". We will cover machine learning potentials in greater detail later in the course, but it is a machine learning model trained to reproduce density functional theory energy, forces, and stresses at a low computational cost. The model we will use in this exercise is called TensorNet-MatPES-r2SCAN from https://arxiv.org/abs/2503.04070.


# Setup


Install the following:

```bash
conda activate cms
uv pip install matcalc[matgl]
```


# Demonstration


First, we will need to define our atomic system. We will study a porous material called a metal-organic framework, specifically MOF-5.


In [1]:
from ase.io import read

atoms = read("mof5.cif")

In [2]:
from ase.visualize import view

view(atoms)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

Now we need to define a calculator. ASE comes with many pre-made calculators (https://ase-lib.org/ase/calculators/calculators.html), and there are many external codes that ship their own ASE calculators.


We will load the TensorNet-MatPES-r2SCAN calculator


In [3]:
from matcalc import load_fp

calc = load_fp("TensorNet-MatPES-r2SCAN-v2025.1-PES")

Now we can attach the calculator to the `Atoms` object.


In [4]:
atoms.calc = calc

We now have everything we need to run a calculation. Let's start with running a static (single-point) calculation.


In [5]:
e_initial = atoms.get_potential_energy()
print(e_initial)


-3339.190185546875


It is always important to understand the units of any given code. In the case of ASE, energies are given in eV (https://ase-lib.org/ase/units.html). There are 96.49 kJ/mol per eV.

An energy on its own is meaningless. It must be referenced to something. We will come back to this.


The structure we loaded not be a minimum energy structure. We must run a geometry optimization to obtain this. We can do this in ASE as well.

ASE comes with many optimizers. We will use BFGS.


In [6]:
from ase.optimize import BFGS

# Set up the BFGS optimizer and specify output file
opt = BFGS(atoms, trajectory="opt.traj")

In [7]:
opt.run(fmax=0.01)  # Run until the maximum force is less than 0.01 eV/Å

      Step     Time          Energy          fmax
BFGS:    0 20:57:15    -3339.190186        0.343955
BFGS:    1 20:57:16    -3339.353516        0.215393
BFGS:    2 20:57:18    -3339.451660        0.134192
BFGS:    3 20:57:20    -3339.477539        0.099504
BFGS:    4 20:57:22    -3339.499512        0.062987
BFGS:    5 20:57:23    -3339.511963        0.051197
BFGS:    6 20:57:24    -3339.516113        0.017635
BFGS:    7 20:57:26    -3339.516602        0.010647
BFGS:    8 20:57:27    -3339.516602        0.009289


np.True_

In [8]:
e_final = atoms.get_potential_energy()  # get the optimized energy
print(e_final - e_initial)  # print the energy difference

-0.326416015625


In [9]:
atoms.get_forces()

array([[-0.00248006, -0.0024826 ,  0.00248047],
       [ 0.00250907,  0.0024982 , -0.00248996],
       [ 0.00249768,  0.00250864,  0.00248022],
       ...,
       [-0.00410385,  0.00584745,  0.00584748],
       [-0.00414311,  0.00580001, -0.00580046],
       [ 0.00410217, -0.00583369,  0.00583553]],
      shape=(424, 3), dtype=float32)

It looks like we found a lower energy structure. Let's take a look at the optimization trajectory now.


In [10]:
trajectory = read("opt.traj", index=":")  # index=":" means read all the frames

In [11]:
from ase.visualize import view

view(trajectory)

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

If you look closely, you will see that the atomic positions updated and the energy went down. The forces went down too.

But note that the unit cell shape and size did not change. In addition to the atomic positions, the unit cell can also be optimized. After all, if you made the unit cell too big, that may not be the lowest energy structure. We can optimize both the positions and unit cell in ASE simultaneously.

In ASE, this is done by wrapping the `Atoms` object in a "Cell Filter". Otherwise, the procedure remains largely the same.


In [12]:
from ase.filters import FrechetCellFilter

# this tells ASE to optimize both atomic positions and cell parameters
fcf = FrechetCellFilter(atoms)


In [ ]:
opt = BFGS(fcf, trajectory="opt2.traj")
opt.run(fmax=0.01)

      Step     Time          Energy          fmax
BFGS:    0 20:57:28    -3339.516602        0.099593
BFGS:    1 20:57:29    -3339.516846        0.099031
BFGS:    2 20:57:30    -3339.522705        0.093071
BFGS:    3 20:57:32    -3339.525879        0.090442
BFGS:    4 20:57:33    -3339.533203        0.086341
BFGS:    5 20:57:34    -3339.540771        0.085061
BFGS:    6 20:57:35    -3339.551270        0.086375
BFGS:    7 20:57:37    -3339.565186        0.089815
BFGS:    8 20:57:38    -3339.586670        0.094765
BFGS:    9 20:57:39    -3339.617676        0.098207
BFGS:   10 20:57:40    -3339.650391        0.101754
BFGS:   11 20:57:42    -3339.684570        0.109160
BFGS:   12 20:57:43    -3339.719238        0.111289
BFGS:   13 20:57:44    -3339.754395        0.108702
BFGS:   14 20:57:45    -3339.788086        0.103835
BFGS:   15 20:57:47    -3339.822510        0.096854
BFGS:   16 20:57:48    -3339.856201        0.088716
BFGS:   17 20:57:49    -3339.888916        0.079592
BFGS:   18 20:

In [ ]:
atoms.get_potential_energy()  # get the optimized energy

In [14]:
view(read("opt2.traj", index=":"))

<Popen: returncode: None args: ['c:\\Users\\asros\\miniconda3\\envs\\cms\\py...>

Finally, we will note that there are many ASE calculators, some based on empirical potentials, some based on ML models, some based on quantum chemistry, and so on. An incomplete list can be found at https://ase-lib.org/ase/calculators/calculators.html. It is easy to swap to a different calculator and to do the same procedure as above.

For instance, we could run a calculation with a Lennard-Jones potential in exactly the same way. Note: Of course, the LJ potential we have chosen here is completely unphysical.


In [15]:
from ase.calculators.lj import LennardJones

In [16]:
# We will run a toy single-point calculation with an LJ potential
# This is completely unphysical because we arbitrarily chosen sigma=1.0 and epsilon=1.0
# But it serves to illustrate how to set up a different calculator and also how
# to specify keyword arguments

atoms.calc = LennardJones(sigma=1.0, epsilon=1.0)
atoms.get_potential_energy()

np.float64(-269.3025268700898)